In [1]:
def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
    ax.tick_params(width=0.5)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def loadRippleFrames(expt):
    try:
        with open(expt.LFPFilePath() + '/' + 'ripple_frames.pkl', 'rb') as f:
            rFrames = pickle.load(f)
    except:
        r_path = [fn for fn in locate('*keep_ripples.npy', expt.LFPFilePath(), ignore=None)][0]
        rtimes = np.load(r_path)
        ts = np.arange(expt.num_frames()) * expt.frame_period()
        ripple_frames = []
        for t in xrange(len(rtimes)):
            ripple_frames.append(find_nearest(ts, rtimes[t][0]))
        ripple_frames = np.asarray(ripple_frames)
        rFrames = ripple_frames[ripple_frames < expt.num_frames()]
        with open(expt.LFPFilePath() + '/' + 'ripple_frames.pkl', 'wb') as f:
            pickle.dump(rFrames, f)
    return rFrames

def ROI_planeID(expt, label):
    signals = expt.imaging_dataset().signals()[label]['rois']
    nROIs = len(signals)
    planeID = []
    for i in xrange(nROIs):
        planeID.append(signals[i]['polygons'][0][0][2])

    return np.asarray(planeID).astype('int')

def resample_trace(fluorescence_trace, number_of_data_points):
    y = fluorescence_trace
    x = np.arange(0, len(y))
    f = interpolate.interp1d(x,y)
    xnew = np.linspace(x.min(), x.max(), num = number_of_data_points)
    ynew = f(xnew)
    return ynew

def get_immobility_frames(expt, immobility_cutoff = 0.2):
    return np.where(expt.velocity()[0] < immobility_cutoff)[0]

In [2]:
def get_delta_anticipatory_licks(expt):
    n_laps = len(expt.behaviorData()['lap'])
    lap_divider = int(n_laps/2)
    time_divider = int(expt.behaviorData()['lap'][lap_divider])
    
    lick_start_times = expt.behaviorData()['licking'][:,0]
    fh_lick_start_times = [lick for lick in lick_start_times if lick > 0 and lick < time_divider]
    sh_lick_start_times = [lick for lick in lick_start_times if lick > time_divider]
    
    normalized_position = pd.DataFrame(expt.behaviorData()['treadmillPosition'], \
                            index = None, columns = ['Time', 'Normalized_Position'])
    
    norm_pos_fh_lts = []
    for lick_time in fh_lick_start_times: 
        index = np.abs(normalized_position['Time'] - lick_time).idxmin()
        norm_pos_fh_lts.append(normalized_position.iloc[index]['Normalized_Position'])
    
    norm_pos_sh_lts = []
    for lick_time in sh_lick_start_times: 
        index = np.abs(normalized_position['Time'] - lick_time).idxmin()
        norm_pos_sh_lts.append(normalized_position.iloc[index]['Normalized_Position'])
        
    reward_info = expt.behaviorData()['__trial_info']['contexts']['reward']
    track_length = expt.behaviorData()['trackLength']
    rz_start = (reward_info['locations'][0] - reward_info['radius']) / track_length
    rz_end = (reward_info['locations'][0] + reward_info['radius']) / track_length

    pzs = (reward_info['locations'][0] - (6 * reward_info['radius'])) / track_length #pre-zone start
    pze = (reward_info['locations'][0] - reward_info['radius']) / track_length #pre-zond end
    
    fh_rz_licks = [pos for pos in norm_pos_fh_lts if pos > rz_start and pos < rz_end]
    fh_other_licks = [pos for pos in norm_pos_fh_lts if pos not in fh_rz_licks]
    fh_pre_zone_licks = [pos for pos in norm_pos_fh_lts if pos > pzs and pos < pze]
    
    if len(fh_other_licks) > 0: 
        first_half_anticip_licks = len(fh_pre_zone_licks)/float(len(fh_other_licks))
    else: 
        first_half_anticip_licks = len(fh_pre_zone_licks)/float(len(fh_rz_licks))

    sh_rz_licks = [pos for pos in norm_pos_sh_lts if pos > rz_start and pos < rz_end]
    sh_other_licks = [pos for pos in norm_pos_sh_lts if pos not in sh_rz_licks]
    sh_pre_zone_licks = [pos for pos in norm_pos_sh_lts if pos > pzs and pos < pze]
    
    if len(sh_other_licks) > 0:
        second_half_anticip_licks = len(sh_pre_zone_licks)/float(len(sh_other_licks))
    else:
        second_half_anticip_licks = len(sh_pre_zone_licks)/float(len(sh_rz_licks))
                
    return second_half_anticip_licks - first_half_anticip_licks

In [3]:
def get_total_RZ_licks(expt):
    lick_start_times = expt.behaviorData()['licking'][:,0] 
    lick_start_times = [lick for lick in lick_start_times if lick > 0]
    normalized_position = pd.DataFrame(expt.behaviorData()['treadmillPosition'], \
                            index = None, columns = ['Time', 'Normalized_Position'])
    
    norm_pos_lts = []
    for lick_time in lick_start_times: 
        index = np.abs(normalized_position['Time'] - lick_time).idxmin()
        norm_pos_lts.append(normalized_position.iloc[index]['Normalized_Position'])
        
    reward_info = expt.behaviorData()['__trial_info']['contexts']['reward']
    track_length = expt.behaviorData()['trackLength']
    rz_start = (reward_info['locations'][0] - reward_info['radius']) / track_length
    rz_end = (reward_info['locations'][0] + reward_info['radius']) / track_length
    
    RZ_licks = [pos for pos in norm_pos_lts if pos > rz_start and pos < rz_end]
    
    return len(RZ_licks)